In [2]:
import torch
import torch.nn.functional as F
import sympy as sp
import pandas as pd
import numpy as np
import numdifftools as nd

In [3]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [4]:
df = pd.read_csv('physics_equations.csv')

functions = []
num_vars_per_func = []

for _, row in df.iterrows():
    formula = row['Formula']
    num_vars = row['# variables']
    function_details = {
        'formula': formula,
        'variables': []
    }
    
    for i in range(1, 11):  
        v_name = row.get(f'v{i}_name', None)
        v_low = row.get(f'v{i}_low', None)
        v_high = row.get(f'v{i}_high', None)
        
        if pd.notna(v_name):
            function_details['variables'].append({
                'name': v_name,
                'low': v_low,
                'high': v_high
            })
    
    functions.append(function_details)
    num_vars_per_func.append(num_vars)

In [5]:
for i, func in enumerate(functions):
    print(f"Function {i+1}:")
    print(f"  Formula: {func['formula']}")
    print(f"  Number of Variables: {num_vars_per_func[i]}")
    print("  Variables:")
    for var in func['variables']:
        print(f"    - Name: {var['name']}, Range: ({var['low']}, {var['high']})")
    print()

Function 1:
  Formula: a*x**2 + b*x + c
  Number of Variables: 3
  Variables:
    - Name: a, Range: (0.1, 1)
    - Name: b, Range: (-1.0, 1.0)
    - Name: c, Range: (-1.0, 1.0)

Function 2:
  Formula: k*x
  Number of Variables: 1
  Variables:
    - Name: k, Range: (-1.0, 1)

Function 3:
  Formula: a*x**2/2
  Number of Variables: 1
  Variables:
    - Name: a, Range: (0.1, 1)

Function 4:
  Formula: G*m1*m2/x**2
  Number of Variables: 3
  Variables:
    - Name: G, Range: (0.1, 1)
    - Name: m1, Range: (0.1, 1.0)
    - Name: m2, Range: (0.1, 1.0)

Function 5:
  Formula: p*g*x
  Number of Variables: 2
  Variables:
    - Name: p, Range: (0.1, 1)
    - Name: g, Range: (0.1, 1.0)

Function 6:
  Formula: (a*x**2 + b*x)/(c*x**2 + d*x + e)
  Number of Variables: 5
  Variables:
    - Name: a, Range: (-1.0, 1)
    - Name: b, Range: (-1.0, 1.0)
    - Name: c, Range: (-1.0, 1.0)
    - Name: d, Range: (-1.0, 1.0)
    - Name: e, Range: (-1.0, 1.0)

Function 7:
  Formula: k*x**3
  Number of Variables:

In [6]:
def generate_function(function, sample_size, x, max_vars, device):
    sympy_symbols = []
    param_tensors = []
    
    for var in function["variables"]:
        sym = sp.symbols(var["name"])
        sympy_symbols.append(sym)
        min_val, max_val = var["low"], var["high"]
        param = (max_val - min_val) * torch.rand(sample_size, 1, device=device) + min_val
        param_tensors.append(param)
    sympy_symbols.append(sp.symbols('x'))
    params = torch.cat(param_tensors, dim=1)
    padded_params = F.pad(params, pad=(0, max_vars - params.size(1)))
    padded_params = padded_params.expand(sample_size, max_vars)
    formula = sp.sympify(function["formula"])
    eval_func = sp.lambdify(sympy_symbols, formula, modules="numpy")
        
    results = []
    for xi in x:
        input_values = torch.cat([params, xi.expand(sample_size, 1)], dim=1)
        results.append(eval_func(*input_values.T))
    results = torch.stack(results, dim=1)
    
    return [results, formula, sympy_symbols, padded_params]